In [ ]:
import geopandas as gpd
import pandas as pd
from scipy.stats import norm 
import numpy as np
from shapely.geometry import Polygon
import string
from sklearn.cluster import AgglomerativeClustering
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import os

In [ ]:
list_char=list(string.ascii_uppercase)
alphabet=list_char+[x+y for x in list_char for y in list_char]
name_columns = alphabet[0:26]

In [ ]:
size_cell = 3

In [ ]:
toy_example = pd.read_csv('../data/toy_example/toy_example_indicators.csv', names= name_columns)
toy_example.index = name_columns

In [ ]:
toy_example_1 = toy_example * 0.3
toy_example_2 = toy_example * 0.7

In [ ]:
# Generation of a grid.
def create_grid_geometry(city_mix, size_cell, name_columns):

    N_rows = len(city_mix)
    polygons = []
    polygon_id = []

    for x in range(N_rows):
        for y in range(N_rows):
            polygons.append(Polygon([(size_cell*x,size_cell*y), 
                                    (size_cell*(x+1), size_cell*y), 
                                    (size_cell*(x+1), size_cell*(y+1)),
                                    (size_cell*x, size_cell*(y+1))]))
            polygon_id.append(name_columns[x] + '_' + name_columns[y])

    grid = gpd.GeoDataFrame({'polygon_id':polygon_id,'geometry':polygons})
    
    centroids = grid.copy()
    centroids['geometry'] = grid.centroid

    return grid, centroids

In [ ]:
grid, centroids = create_grid_geometry(toy_example, size_cell, name_columns)

In [ ]:
def add_demographics(demographics, grid):

    demographics_stacked = demographics.T.stack().reset_index()
    demographics_stacked = demographics_stacked.rename(columns = {0:'res_mix'})
    demographics_stacked['polygon_id'] = demographics_stacked['level_0'] + '_' + demographics_stacked['level_1']
    demographics_stacked = demographics_stacked.drop(columns = ['level_0','level_1'])

    return grid.merge(demographics_stacked, on = 'polygon_id')

In [ ]:
toy_example_1 = add_demographics(toy_example_1, grid)
toy_example_2 = add_demographics(toy_example_2, grid)

In [ ]:
def create_mask(grid, distance):

    grid_mask = grid.copy()
    grid_mask['geometry'] = grid_mask['geometry'].centroid.buffer(distance)
    grid_mask = grid_mask.drop(columns = 'res_mix').rename(columns ={'polygon_id':'from_polygon_id'})

    grid_target = grid.copy()
    grid_target['geometry'] = grid_target['geometry'].centroid
    grid_target = grid_target.rename(columns ={'polygon_id':'to_polygon_id'})

    grid_mask = grid_mask.sjoin(grid_target).drop(columns = 'index_right')
    grid_mask = grid_mask.reset_index(drop = True)

    return grid_mask


In [ ]:
def compute_shortest_paths(res_mix,size_cell, grid):
    # Computing the shortest paths from cell to cell.
    grid = grid.copy()
    grid['geometry'] = grid['geometry'].centroid
    shortest_paths = pd.DataFrame({'from_polygon_id':[],'to_polygon_id':[],'distance':[]})

    for i in res_mix['polygon_id']:
        shortest_paths_i = pd.DataFrame({'from_polygon_id':[i]*len(res_mix),
                                         'to_polygon_id':res_mix['polygon_id'],
                                         'distance':grid.distance(grid.loc[res_mix['polygon_id'] == i,'geometry'].values[0])})
        shortest_paths = pd.concat([shortest_paths,shortest_paths_i], ignore_index=True)
    
    shortest_paths['weight'] = 1/(0.3*size_cell*0.707)**2
    shortest_paths['weight'] = shortest_paths['weight'].mask(shortest_paths['distance'] > 0,
                                                             1/(0.3*shortest_paths['distance'])**2)

    return shortest_paths

In [ ]:
shortest_paths = compute_shortest_paths(toy_example_1,size_cell, grid)

In [ ]:
def compute_exposure(toy_example,r,size_cell):
    
    toy_example = toy_example.copy()
    grid_mask = create_mask(toy_example, size_cell*r)

    weight_Petrovic = grid_mask[['from_polygon_id','to_polygon_id']].groupby('from_polygon_id').count().reset_index()
    weight_Petrovic = weight_Petrovic.rename(columns = {'to_polygon_id':'weight_Petrovic'})
    exposure_Petrovic = grid_mask.groupby('from_polygon_id').mean().reset_index()
    exposure_Petrovic = exposure_Petrovic.merge(weight_Petrovic, on ='from_polygon_id')
    exposure_Petrovic = exposure_Petrovic.rename(columns = {'from_polygon_id':'polygon_id','res_mix':'exposure_Petrovic'})

    grid_mask_Lan = grid_mask.merge(shortest_paths.drop(columns = 'weight'), on = ['from_polygon_id','to_polygon_id'])
    grid_mask_Lan['weight'] = 1
    grid_mask_Lan['weight'] = grid_mask_Lan['weight'].mask(grid_mask_Lan['distance'] > 0, 
                                                        (1- (grid_mask_Lan['distance']/size_cell/r)**2) ** 2)
    grid_mask_Lan['exposure'] = grid_mask_Lan['weight'] * grid_mask['res_mix'] 
    exposure_Lan = grid_mask_Lan.groupby('from_polygon_id').sum().reset_index()
    exposure_Lan['exposure'] = exposure_Lan['exposure'] / exposure_Lan['weight']
    exposure_Lan = exposure_Lan.rename(columns = {'from_polygon_id':'polygon_id','exposure':'exposure_Lan','weight':'weight_Lan'})
    toy_example = toy_example.merge(exposure_Lan[['polygon_id','exposure_Lan','weight_Lan']], on = 'polygon_id')
    toy_example = toy_example.merge(exposure_Petrovic[['polygon_id','exposure_Petrovic','weight_Petrovic']], on = 'polygon_id')


    return toy_example

In [ ]:
def compute_entropy(table, column):
    table['entropy_' + column] = 0
    table['entropy_' + column] = table['entropy_' + column].mask((table[column] > 0) & (table[column] < 1),
                                                                  table[column] * np.log2(table[column]) + 
                                                                  (1-table[column]) * np.log2(1 - table[column]))
    return table

In [ ]:
r = 3
exposure = compute_exposure(toy_example_1,r,size_cell)

In [ ]:
res_mix_city = exposure['res_mix'].sum()/len(exposure)
entropy_city = -(res_mix_city * np.log2(res_mix_city) + (1 - res_mix_city) * np.log2((1 - res_mix_city)))

exposure = compute_entropy(exposure, 'res_mix')
entropy_res_mix = 1 + exposure['entropy_res_mix'].sum()/len(exposure)/entropy_city

exposure = compute_entropy(exposure, 'exposure_Petrovic')
entropy_Petrovic = 1 + exposure['entropy_exposure_Petrovic'].sum()/len(exposure)/entropy_city

exposure = compute_entropy(exposure, 'exposure_Lan')
entropy_Lan = 1 + exposure['entropy_exposure_Lan'].sum()/len(exposure)/entropy_city

In [ ]:
evolution_1 = pd.DataFrame({'distance':[],'entropy_Petrovic':[],'entropy_Lan':[]})
evolution_2 = pd.DataFrame({'distance':[],'entropy_Petrovic':[],'entropy_Lan':[]})

for r in range(1,8):

    exposure = compute_exposure(toy_example_1,r,size_cell)
    res_mix_city = exposure['res_mix'].sum()/len(exposure)
    entropy_city = -(res_mix_city * np.log2(res_mix_city) + (1 - res_mix_city) * np.log2((1 - res_mix_city)))

    exposure = compute_entropy(exposure, 'res_mix')
    entropy_res_mix = 1 + exposure['entropy_res_mix'].sum()/len(exposure)/entropy_city

    exposure = compute_entropy(exposure, 'exposure_Petrovic')
    entropy_Petrovic = 1 + exposure['entropy_exposure_Petrovic'].sum()/len(exposure)/entropy_city

    exposure = compute_entropy(exposure, 'exposure_Lan')
    entropy_Lan = 1 + exposure['entropy_exposure_Lan'].sum()/len(exposure)/entropy_city

    evolution_1 = pd.concat([evolution_1,
                             pd.DataFrame({'distance':[r],
                                           'entropy_Petrovic':[entropy_Petrovic],
                                           'entropy_Lan':[entropy_Lan]})], ignore_index = True)


for r in range(1,8):

    exposure = compute_exposure(toy_example_2,r,size_cell)
    res_mix_city = exposure['res_mix'].sum()/len(exposure)
    entropy_city = -(res_mix_city * np.log2(res_mix_city) + (1 - res_mix_city) * np.log2((1 - res_mix_city)))

    exposure = compute_entropy(exposure, 'res_mix')
    entropy_res_mix = 1 + exposure['entropy_res_mix'].sum()/len(exposure)/entropy_city

    exposure = compute_entropy(exposure, 'exposure_Petrovic')
    entropy_Petrovic = 1 + exposure['entropy_exposure_Petrovic'].sum()/len(exposure)/entropy_city

    exposure = compute_entropy(exposure, 'exposure_Lan')
    entropy_Lan = 1 + exposure['entropy_exposure_Lan'].sum()/len(exposure)/entropy_city

    evolution_2 = pd.concat([evolution_2,
                             pd.DataFrame({'distance':[r],
                                           'entropy_Petrovic':[entropy_Petrovic],
                                           'entropy_Lan':[entropy_Lan]})], ignore_index = True)
    


In [ ]:
fig, ax = plt.subplots(1,2,figsize=(8, 4), layout = 'constrained')

# Fine grid
ax[0].plot(evolution_1['distance'], evolution_1['entropy_Petrovic'], label = 'City 1')
ax[0].plot(evolution_2['distance'],evolution_2['entropy_Petrovic'], label = 'City 2')
ax[0].set_ylim(0,evolution_2['entropy_Lan'].max())
ax[0].set_xlabel('Radius of the local environment')
ax[0].set_ylabel('Information index using the method of Petrovic')
ax[0].legend()

ax[1].plot(evolution_1['distance'],evolution_1['entropy_Lan'], label = 'City 1')
ax[1].plot(evolution_2['distance'],evolution_2['entropy_Lan'], label = 'City 2')
ax[1].set_xlabel('Radius of the local environment')
ax[1].set_ylabel('Information index using the method of Lan')
ax[1].legend()

plt.close()

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(8, 3.2), layout = 'constrained')

toy_example_1.plot(ax=ax[0], cmap = 'Blues', column = 'res_mix', vmin = 0, vmax = 1)
toy_example_1.plot(ax=ax[0],color = 'none', edgecolor = 'black')
toy_example_2.plot(ax=ax[1], cmap = 'Blues', column = 'res_mix', vmin = 0, vmax = 1)
toy_example_2.plot(ax=ax[1],color = 'none', edgecolor = 'black')

# Set titles
ax[0].set_title('City 1', size = 15)
ax[1].set_title('City 2', size = 15)
ax[0].axis('off')
ax[1].axis('off')

fig.colorbar(cm.ScalarMappable(norm = mpl.colors.Normalize(vmin=-0, vmax=1), cmap='Blues'),
                               ax=ax[1], 
                               shrink=1).set_label(label='Group share',size=12)
#grid_hh_data.plot(ax=ax[1],color = 'none', edgecolor = 'black')

# ax[0].plot(evolution_1['distance'], evolution_1['entropy_Petrovic'], label = 'City 1')
# ax[0].plot(evolution_2['distance'],evolution_2['entropy_Petrovic'], label = 'City 2')
# ax[0].set_ylim(0,evolution_2['entropy_Lan'].max())
# ax[0].set_xlabel('Radius of the local environment')
# ax[0].set_ylabel('Information index using the method of Petrovic')
# ax[0].legend()

# ax[1].plot(evolution_1['distance'],evolution_1['entropy_Lan'], label = 'City 1')
# ax[1].plot(evolution_2['distance'],evolution_2['entropy_Lan'], label = 'City 2')
# ax[1].set_xlabel('Radius of the local environment')
# ax[1].set_ylabel('Information index using the method of Lan')
# ax[1].legend()

plt.close()